In [1]:
import pandas as pd, numpy as np, glob
from Bio.Seq import Seq

import warnings
from Bio import BiopythonWarning
warnings.simplefilter('ignore', BiopythonWarning)

def makeorf(df,fill='.'):
    ref = Seq(''.join(df[(df.Ref!='.')]['Ref'].tolist()))
    alt = Seq(''.join(df[(df.Alt!='.')]['Alt'].tolist()))
    
    if df.Strand.min() < 0:
        ref = ref.reverse_complement()
        alt = alt.reverse_complement()
        
    return ref,alt

In [2]:
## Bring in chrommap
chrommap = pd.read_csv('/home/croth/Downloads/B3502/DATA/chrommap.csv.gz')
chrommap['Seqid'] = chrommap['Contig']
chrommap['Chromosome'] = chrommap.index+1

In [3]:
## Bring in GFF file
gffpath ='/home/croth/Downloads/B3502/REF/FungiDB-48_CneoformansJEC21.gff.gz'

names = ["Seqid", "Source", "Type", 
         "Start", "End", "Score", 
         "Strand", "Phase", "Attributes"]

descriptions = ['hypothetical protein','unspecified product']
dtype = ["str","str","str","int","int","str","str","str","str"]

gff = pd.read_csv(gffpath,
                   comment="#",header=None,
                  delimiter="\t",names=names,
                    dtype=dict(zip(names,dtype)))

foi = ['gene','three_prime_UTR','five_prime_UTR','CDS']

gff = gff[(gff.Type.isin(foi))]

gff['Parent'] = [a.split('Parent=')[-1].split(';')[0].split('ID=')[-1] 
                 for a in gff.Attributes ]

gff['Gene'] = [a.split(';')[0].split('D=')[-1].split('-')[0]
               for a in gff.Attributes]

genes = gff[(gff.Type=='gene')].copy()
genes['Description'] = [a.split('description=')[-1].split('%2C')[0] 
                        for a in genes.Attributes]

genes = genes.merge(chrommap[['Seqid','Length','Chromosome','Midpts','Cumlen']])
genes.head()

,Seqid,Source,Type,Start,End,Score,Strand,Phase,Attributes,Parent,Gene,Description,Length,Chromosome,Midpts,Cumlen
0,AE017352.1,VEuPathDB,gene,713524,714773,.,-,.,ID=CNL06190;description=aldo-keto reductase%2C...,CNL06190,CNL06190,aldo-keto reductase,906719,12,17047869.5,16594510
1,AE017352.1,VEuPathDB,gene,85905,87112,.,-,.,ID=CNL03960;description=large subunit ribosoma...,CNL03960,CNL03960,large subunit ribosomal protein,906719,12,17047869.5,16594510
2,AE017352.1,VEuPathDB,gene,832997,835677,.,+,.,ID=CNL06580;description=hypothetical protein,CNL06580,CNL06580,hypothetical protein,906719,12,17047869.5,16594510
3,AE017352.1,VEuPathDB,gene,361800,362947,.,-,.,ID=CNL04850;description=HDEL sequence binding ...,CNL04850,CNL04850,HDEL sequence binding protein,906719,12,17047869.5,16594510
4,AE017352.1,VEuPathDB,gene,531422,533575,.,+,.,ID=CNL05490;description=endopeptidase%2C putative,CNL05490,CNL05490,endopeptidase,906719,12,17047869.5,16594510


In [4]:
B3502 = sorted(glob.glob('../GENES/B3502*/*.csv.gz')) 
CF830 = sorted(glob.glob('../GENES/CF830/*.csv.gz'))
JEC21 = sorted(glob.glob('../GENES/JEC21/*.csv.gz'))
#JEC20 = sorted(glob.glob('../GENES/JEC20/*.csv.gz'))
samplespath = B3502+CF830+JEC21
samplespath[:5],len(samplespath)

(['../GENES/B3502_A1/CNA00180-t26_1.csv.gz',
  '../GENES/B3502_A1/CNA00180-t26_2.csv.gz',
  '../GENES/B3502_A1/CNA01990-t26_1.csv.gz',
  '../GENES/B3502_A1/CNA01990-t26_2.csv.gz',
  '../GENES/B3502_A1/CNA02430-t26_1.csv.gz'],
 765)

In [5]:
todf = []
for s in samplespath:

    sample = s.split('/')[2]
    genep = s.split('/')[-1].split('.csv')[0]
    gene_name = s.split('/')[-1].split('-t26')[0]

    temp = pd.read_csv(s)
    gene = temp.Gene.min()
    assert sample == temp.Sample.min()
    assert gene == genep
        
    cds = temp[(temp.Type==0)]
        
    ref,alt = makeorf(cds)
        
    el = cds.Pos.unique().shape[0]/3-1
        
    ra = ref.translate(to_stop=True)
    aa = alt.translate(to_stop=True)
        
    rl = len(ra)
    al = len(aa)
        
    sr = ref.translate().count('*')
    sa = alt.translate().count('*')
        
    ns = sa - 1
    for i in range(np.min([len(ra),len(aa)])):
        if ra[i]!=aa[i]:
            ns = ns + 1
            
    nvars = temp[(temp.Isvar==1)].shape[0]
    utr3 = temp[(temp.Type==3) & (temp.Isvar==1)].shape[0]
    utr5 = temp[(temp.Type==5) & (temp.Isvar==1)].shape[0]
    inexon = temp[(temp.Type==0) & (temp.Isvar==1)].shape[0]
    inintron = temp[(temp.Type==-1) & (temp.Isvar==1)].shape[0]
        
    todf.append((sample,gene_name,gene,
                 el,rl,al,sr,sa,ns,
                 nvars,utr5,utr3,inexon,inintron
                ))

In [6]:
resdf = pd.DataFrame(todf,
        columns=['Strain','Gene','Parent','Expected',
                 'Ref','Alt','Refstop','Altstop','Nonsyn',
                 'Nvars','Utr5','Utr3','Exon','Intron'
                ])
resdf.tail()

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron
760,JEC21,CNM01730,CNM01730-t26_2,725.0,725,725,1,1,0,0,0,0,0,0
761,JEC21,CNN00400,CNN00400-t26_1,1379.0,1379,1379,1,1,0,0,0,0,0,0
762,JEC21,CNN01270,CNN01270-t26_1,691.0,691,691,1,1,0,0,0,0,0,0
763,JEC21,CNN01450,CNN01450-t26_1,297.0,297,297,1,1,0,1,0,0,0,1
764,JEC21,CNN01790,CNN01790-t26_1,277.0,277,277,1,1,0,0,0,0,0,0


In [7]:
resdf.Gene.unique().shape

(145,)

In [8]:
resdf[(resdf.Nonsyn>0)].Gene.unique().shape

(67,)

In [9]:
resdf[(resdf.Utr5>0)].shape

(121, 14)

In [10]:
resdf[(resdf.Utr3>0)].shape

(160, 14)

In [11]:
resdf.shape

(765, 14)

In [12]:
resdf[(resdf.Altstop==0)].shape

(4, 14)

In [13]:
resdf[(resdf.Altstop>1)].shape

(14, 14)

In [14]:
resdf[(resdf.Refstop==0)].shape

(0, 14)

In [15]:
resdf[(resdf.Refstop>1)].shape

(0, 14)

In [16]:
toredo = resdf[(resdf.Expected!=resdf.Ref)]
toredo.shape

(0, 14)

In [17]:
resdf.Strain.unique().shape

(5,)

In [18]:
## Genes with stop gains within them
sorted(resdf[(resdf.Altstop>1)].Gene.unique())

['CNA05300', 'CNB00130', 'CNF02550', 'CNI02965', 'CNN01270']

In [19]:
resdf[(resdf.Altstop!=1)].sort_values('Gene')

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron
12,B3502_A1,CNA05300,CNA05300-t26_1,1271.0,1271,516,1,2,1,1,0,0,1,0
21,B3502_A1,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4
174,B3502_B1,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4
327,B3502_B7,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4
560,CF830,CNF02550,CNF02550-t26_1,556.0,556,240,1,2,1,1,0,0,1,0
586,CF830,CNI02965,CNI02965-t26_2,189.0,189,132,1,3,10,5,0,0,5,0
585,CF830,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,10,5,0,0,5,0
433,B3502_B7,CNI02965,CNI02965-t26_2,189.0,189,132,1,3,9,4,0,0,4,0
432,B3502_B7,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,9,4,0,0,4,0
739,JEC21,CNI02965,CNI02965-t26_2,189.0,189,188,1,0,61,5,0,0,5,0


In [20]:
## CNF02550 hypothetical 

In [21]:
ric8_path = '../GENES/JEC21/CNN01270-t26_1.csv.gz'
ric8 = pd.read_csv(ric8_path,index_col=0)

In [22]:
ric8_name = 'CNN01270'
resdf[(resdf.Gene==ric8_name) &(resdf.Altstop>1)]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron
150,B3502_A1,CNN01270,CNN01270-t26_1,691.0,691,456,1,2,1,1,0,0,1,0
303,B3502_B1,CNN01270,CNN01270-t26_1,691.0,691,456,1,2,1,1,0,0,1,0
456,B3502_B7,CNN01270,CNN01270-t26_1,691.0,691,456,1,2,1,1,0,0,1,0


In [23]:
resdf[(resdf.Strain=='CF830') & (resdf.Altstop>1)]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron
560,CF830,CNF02550,CNF02550-t26_1,556.0,556,240,1,2,1,1,0,0,1,0
585,CF830,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,10,5,0,0,5,0
586,CF830,CNI02965,CNI02965-t26_2,189.0,189,132,1,3,10,5,0,0,5,0


In [24]:
finalres = resdf.merge(genes[['Gene','Chromosome','Seqid',
                   'Start','End','Strand','Description']])

In [25]:
finalres.head()

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
0,B3502_A1,CNA00180,CNA00180-t26_1,599.0,599,599,1,1,0,3,0,0,0,3,1,AE017341.1,54488,57404,-,vacuolar calcium exchanger
1,B3502_A1,CNA00180,CNA00180-t26_2,606.0,606,606,1,1,0,3,0,0,0,3,1,AE017341.1,54488,57404,-,vacuolar calcium exchanger
2,B3502_B1,CNA00180,CNA00180-t26_1,599.0,599,599,1,1,0,3,0,0,0,3,1,AE017341.1,54488,57404,-,vacuolar calcium exchanger
3,B3502_B1,CNA00180,CNA00180-t26_2,606.0,606,606,1,1,0,3,0,0,0,3,1,AE017341.1,54488,57404,-,vacuolar calcium exchanger
4,B3502_B7,CNA00180,CNA00180-t26_1,599.0,599,599,1,1,0,1,0,0,0,1,1,AE017341.1,54488,57404,-,vacuolar calcium exchanger


In [26]:
altstoped = finalres[(finalres.Altstop>1)]
altstoped.shape

(14, 20)

In [27]:
altstoped

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
60,B3502_A1,CNA05300,CNA05300-t26_1,1271.0,1271,516,1,2,1,1,0,0,1,0,1,AE017341.1,1405078,1410006,+,putative chitin synthase
105,B3502_A1,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4,2,AE017342.1,37214,40541,-,caspase
106,B3502_B1,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4,2,AE017342.1,37214,40541,-,caspase
107,B3502_B7,CNB00130,CNB00130-t26_1,647.0,647,118,1,41,51,30,9,2,15,4,2,AE017342.1,37214,40541,-,caspase
508,CF830,CNF02550,CNF02550-t26_1,556.0,556,240,1,2,1,1,0,0,1,0,6,AE017346.1,738808,741219,+,expressed protein
630,B3502_A1,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,10,5,0,0,5,0,9,AE017349.1,806743,808487,-,hypothetical protein
631,B3502_A1,CNI02965,CNI02965-t26_2,189.0,189,132,1,3,10,5,0,0,5,0,9,AE017349.1,806743,808487,-,hypothetical protein
634,B3502_B7,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,9,4,0,0,4,0,9,AE017349.1,806743,808487,-,hypothetical protein
635,B3502_B7,CNI02965,CNI02965-t26_2,189.0,189,132,1,3,9,4,0,0,4,0,9,AE017349.1,806743,808487,-,hypothetical protein
636,CF830,CNI02965,CNI02965-t26_1,189.0,189,132,1,3,10,5,0,0,5,0,9,AE017349.1,806743,808487,-,hypothetical protein


In [28]:
finalres.to_csv('../GENES/gene_changes.csv.gz',index=None)
finalres.shape

(765, 20)

In [29]:
finalres[(finalres.Gene=='CNM00880')]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
715,B3502_A1,CNM00880,CNM00880-t26_1,343.0,343,343,1,1,0,0,0,0,0,0,13,AE017353.1,256057,258338,-,DNA polymerase processivity factor
716,B3502_B1,CNM00880,CNM00880-t26_1,343.0,343,343,1,1,0,3,0,3,0,0,13,AE017353.1,256057,258338,-,DNA polymerase processivity factor
717,B3502_B7,CNM00880,CNM00880-t26_1,343.0,343,343,1,1,0,0,0,0,0,0,13,AE017353.1,256057,258338,-,DNA polymerase processivity factor
718,CF830,CNM00880,CNM00880-t26_1,343.0,343,343,1,1,0,0,0,0,0,0,13,AE017353.1,256057,258338,-,DNA polymerase processivity factor
719,JEC21,CNM00880,CNM00880-t26_1,343.0,343,343,1,1,0,0,0,0,0,0,13,AE017353.1,256057,258338,-,DNA polymerase processivity factor


In [30]:
finalres[(finalres.Gene=='CNJ01150')]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
655,B3502_A1,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
656,B3502_B1,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,1,0,0,1,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
657,B3502_B7,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
658,CF830,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
659,JEC21,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein


In [31]:
finalres[(finalres.Gene=='CNF04940')]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
530,B3502_A1,CNF04940,CNF04940-t26_1,526.0,526,526,1,1,0,0,0,0,0,0,6,AE017346.1,1437068,1438867,-,hypothetical protein
531,B3502_B1,CNF04940,CNF04940-t26_1,526.0,526,526,1,1,1,12,0,1,11,0,6,AE017346.1,1437068,1438867,-,hypothetical protein
532,B3502_B7,CNF04940,CNF04940-t26_1,526.0,526,526,1,1,0,0,0,0,0,0,6,AE017346.1,1437068,1438867,-,hypothetical protein
533,CF830,CNF04940,CNF04940-t26_1,526.0,526,526,1,1,0,0,0,0,0,0,6,AE017346.1,1437068,1438867,-,hypothetical protein
534,JEC21,CNF04940,CNF04940-t26_1,526.0,526,526,1,1,0,0,0,0,0,0,6,AE017346.1,1437068,1438867,-,hypothetical protein


In [32]:
finalres[(finalres.Gene=='CNJ01150')]

,Strain,Gene,Parent,Expected,Ref,Alt,Refstop,Altstop,Nonsyn,Nvars,Utr5,Utr3,Exon,Intron,Chromosome,Seqid,Start,End,Strand,Description
655,B3502_A1,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
656,B3502_B1,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,1,0,0,1,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
657,B3502_B7,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
658,CF830,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein
659,JEC21,CNJ01150,CNJ01150-t26_1,763.0,763,763,1,1,0,0,0,0,0,0,10,AE017350.1,322864,325698,-,putative GTPase-activating protein


In [33]:
#test = pd.read_csv('../DATA/OLD/gene_changes.csv.gz')
#test.shape

In [34]:
#test[(test.Strain=='Stock6') & (test.Altstop>1)]

In [35]:
#test[(test.Strain=='CF830') & (resdf.Altstop>1)]

In [36]:
#test[(test.Gene==ric8_name)]

In [37]:
#a1 = pd.read_csv('../GENES/B3502_A1_Stock1/CNN01270-t26_1.csv.gz',
#                 index_col=0)
#a1[(a1.Isvar==1)]

In [38]:
#d1 = pd.read_csv('../GENES/B3502_D1_Stock6/CNN01270-t26_1.csv.gz',
#                 index_col=0)
#d1[(d1.Isvar==1)]